# Docugami
This notebook covers how to load documents from `Docugami`. It provides the advantages of using this system over alternative data loaders.

## Prerequisites
1. Install necessary python packages.
2. Grab an access token for your workspace, and make sure it is set as the `DOCUGAMI_API_KEY` environment variable.
3. Grab some docset and document IDs for your processed documents, as described here: https://help.docugami.com/home/docugami-api

In [21]:
# You need the dgml-utils package to use the DocugamiLoader (run pip install directly without "poetry run" if you are not using poetry)
!poetry run pip install --upgrade dgml-utils --quiet

## Quick start

1. Create a [Docugami workspace](http://www.docugami.com) (free trials available)
2. Add your documents (PDF, DOCX or DOC) and allow Docugami to ingest and cluster them into sets of similar documents, e.g. NDAs, Lease Agreements, and Service Agreements. There is no fixed set of document types supported by the system, the clusters created depend on your particular documents, and you can [change the docset assignments](https://help.docugami.com/home/working-with-the-doc-sets-view) later.
3. Create an access token via the Developer Playground for your workspace. [Detailed instructions](https://help.docugami.com/home/docugami-api)
4. Explore the [Docugami API](https://api-docs.docugami.com) to get a list of your processed docset IDs, or just the document IDs for a particular docset. 
6. Use the DocugamiLoader as detailed below, to get rich semantic chunks for your documents.
7. Optionally, build and publish one or more [reports or abstracts](https://help.docugami.com/home/reports). This helps Docugami improve the semantic XML with better tags based on your preferences, which are then added to the DocugamiLoader output as metadata. Use techniques like [self-querying retriever](/docs/modules/data_connection/retrievers/self_query/) to do high accuracy Document QA.

## Advantages vs Other Chunking Techniques

Appropriate chunking of your documents is critical for retrieval from documents. Many chunking techniques exist, including simple ones that rely on whitespace and recursive chunk splitting based on character length. Docugami offers a different approach:

1. **Intelligent Chunking:** Docugami breaks down every document into a hierarchical semantic XML tree of chunks of varying sizes, from single words or numerical values to entire sections. These chunks follow the semantic contours of the document, providing a more meaningful representation than arbitrary length or simple whitespace-based chunking.
2. **Structured Representation:** In addition, the XML tree indicates the structural contours of every document, using attributes denoting headings, paragraphs, lists, tables, and other common elements, and does that consistently across all supported document formats, such as scanned PDFs or DOCX files. It appropriately handles long-form document characteristics like page headers/footers or multi-column flows for clean text extraction.
3. **Semantic Annotations:** Chunks are annotated with semantic tags that are coherent across the document set, facilitating consistent hierarchical queries across multiple documents, even if they are written and formatted differently. For example, in set of lease agreements, you can easily identify key provisions like the Landlord, Tenant, or Renewal Date, as well as more complex information such as the wording of any sub-lease provision or whether a specific jurisdiction has an exception section within a Termination Clause.
4. **Additional Metadata:** Chunks are also annotated with additional metadata, if a user has been using Docugami. This additional metadata can be used for high-accuracy Document QA without context window restrictions. See detailed code walk-through below.


In [2]:
import os
from langchain.document_loaders import DocugamiLoader

## Load Documents

If the DOCUGAMI_API_KEY environment variable is set, there is no need to pass it in to the loader explicitly otherwise you can pass it in as the `access_token` parameter.

The DocugamiLoader has a default minimum chunk size of 32. Chunks smaller than that are appended to subsequent chunks. Set min_chunk_size to 0 to get all structural chunks regardless of size.

In [3]:
DOCUGAMI_API_KEY = os.environ.get("DOCUGAMI_API_KEY")

docset_id = "26xpy3aes7xp"
document_ids = ["d7jqdzcj50sj", "cgd1eacfkchw"]

# To load all docs in the given docset ID, just don't provide document_ids
loader = DocugamiLoader(docset_id=docset_id, document_ids=document_ids)
docs = loader.load()
docs

[Document(page_content='MASTER SERVICES AGREEMENT This Services Agreement (the “Agreement”) sets forth terms under which MagicSoft, Inc. a Washington Corporation (“Company”) located at 600 4th Ave , Seattle , WA 98104 shall provide services to Daltech, Inc. , a Washington Corporation (the “Client”) located at 701 1st St , Kirkland , WA 98033 . This Agreement is effective as of February 15, 2021 (“ Effective Date ”).', metadata={'xpath': '/dg:chunk/docset:MASTERSERVICESAGREEMENT-section/docset:MASTERSERVICESAGREEMENT/dg:chunk[1]/docset:ThisServicesAgreement', 'id': 'cgd1eacfkchw', 'name': 'Master Services Agreement - Daltech.docx', 'source': 'Master Services Agreement - Daltech.docx', 'structure': 'h1 p', 'tag': 'chunk ThisServicesAgreement', 'Liability': '', 'Workers Compensation Insurance': '$1,000,000', 'Limit': '$1,000,000', 'Commercial General Liability Insurance': '$2,000,000', 'Technology Professional Liability Errors Omissions Policy': '$5,000,000', 'Excess Liability Umbrella Co

The `metadata` for each `Document` (really, a chunk of an actual PDF, DOC or DOCX) contains some useful additional information:

1. **id and source:** ID and Name of the file (PDF, DOC or DOCX) the chunk is sourced from within Docugami.
2. **xpath:** XPath inside the XML representation of the document, for the chunk. Useful for source citations directly to the actual chunk inside the document XML.
3. **structure:** Structural attributes of the chunk, e.g. h1, h2, div, table, td, etc. Useful to filter out certain kinds of chunks if needed by the caller.
4. **tag:** Semantic tag for the chunk, using various generative and extractive techniques. More details here: https://github.com/docugami/DFM-benchmarks

## Basic Use: Docugami Loader for Document QA

You can use the Docugami Loader like a standard loader for Document QA over multiple docs, albeit with much better chunks that follow the natural contours of the document. There are many great tutorials on how to do this, e.g. [this one](https://www.youtube.com/watch?v=3yPBVii7Ct0). We can just use the same code, but use the `DocugamiLoader` for better chunking, instead of loading text or PDF files directly with basic splitting techniques.

In [ ]:
!poetry run pip -q install openai tiktoken chromadb

In [15]:
# For this example, we already have a processed docset for a set of lease documents
loader = DocugamiLoader(docset_id="zo954yqy53wp")
documents = loader.load()
documents

[Document(page_content='OFFICE LEASE THIS OFFICE LEASE (the "Lease") is made and entered into as of March 29th , 2019 , by and between Landlord and Tenant. "Date of this Lease" shall mean the date on which the last one of the Landlord and Tenant has signed this Lease.', metadata={'xpath': '/docset:OFFICELEASE-section/docset:OFFICELEASE-section/dg:chunk/docset:Lease', 'id': 'cpzwzcurck2t', 'name': 'Sample Commercial Leases/Shorebucks LLC_AZ.pdf', 'source': 'Sample Commercial Leases/Shorebucks LLC_AZ.pdf', 'structure': 'h1 h1 p', 'tag': 'chunk Lease', 'Lease Date': 'March  29th , 2019', 'Landlord': 'Menlo Group', 'Tenant': 'Shorebucks LLC'}),
 Document(page_content='W I T N E S S E T H Subject to and on the terms and conditions of this Lease, Landlord leases to Tenant and Tenant hires from Landlord the Premises.', metadata={'xpath': '/docset:OFFICELEASE-section/docset:OFFICELEASE-section/docset:WITNESSETH-section/docset:WITNESSETH/dg:chunk/docset:TheTerms', 'id': 'cpzwzcurck2t', 'name': 

The documents returned by the loader are already split, so we don't need to use a text splitter. Optionally, we can use the metadata on each document, for example the structure or tag attributes, to do any post-processing we want.

We will just use the output of the `DocugamiLoader` as-is to set up a retrieval QA chain the usual way.

In [20]:
# We will use chromadb as a vector store
!poetry run pip install --upgrade chromadb --quiet

In [32]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import chromadb

embedding = OpenAIEmbeddings()
persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("docugami_test")
vectordb = Chroma.from_documents(documents=documents, embedding=embedding)
retriever = vectordb.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True
)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_20678/2215160652.py", line 5, in <module>
    import chromadb
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/chromadb/__init__.py", line 4, in <module>
    import chromadb.config
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/chromadb/config.py", line 1, in <module>
    from pydantic import BaseSettings
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/pydantic/__init__.py", line 218, in __getattr__
    return _getattr_migration(attr_name)
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/pydantic/_migration.py", line 294, in wrapper
    

In [24]:
# Try out the retriever with an example query
qa_chain("What can tenants do with signage on their properties?")

{'query': 'What can tenants do with signage on their properties?',
 'result': ' Tenants can place or attach signs (digital or otherwise) to their properties after receiving written permission from the Landlord, which permission shall not be unreasonably withheld. The signs must conform to all applicable laws, ordinances, etc. governing the same and the Tenant must remove and clean any window or glass identification promptly upon vacating the Premises.',
 'source_documents': [Document(page_content='6.01 Signage. Tenant may place or attach to the Premises signs (digital or otherwise) or other such identification as needed after receiving written permission from the Landlord, which permission shall not be unreasonably withheld. Any damage caused to the Premises by the Tenant’s erecting or removing such signs shall be repaired promptly by the Tenant at the Tenant’s expense. Any signs or other form of identification allowed must conform to all applicable laws, ordinances, etc. governing the

## Using Docugami to Add Metadata to Chunks for High Accuracy Document QA

One issue with large documents is that the correct answer to your question may depend on chunks that are far apart in the document. Typical chunking techniques, even with overlap, will struggle with providing the LLM sufficent context to answer such questions. With upcoming very large context LLMs, it may be possible to stuff a lot of tokens, perhaps even entire documents, inside the context but this will still hit limits at some point with very long documents, or a lot of documents.

For example, if we ask a more complex question that requires the LLM to draw on chunks from different parts of the document, even OpenAI's powerful LLM is unable to answer correctly.

In [25]:
chain_response = qa_chain("What is rentable area for the property owned by DHA Group?")
chain_response["result"]  # correct answer should be 13,500 sq ft

' The rentable area for the property owned by DHA Group is not specified in the given context.'

At first glance the answer may seem reasonable, but if you review the source chunks carefully for this answer, you will see that the chunking of the document did not end up putting the Landlord name and the rentable area in the same context, since they are far apart in the document. The retriever therefore ends up finding unrelated chunks from other documents not even related to the **DHA Group** landlord. That landlord happens to be mentioned on the first page of the file **Shorebucks LLC_NJ.pdf** file, and while one of the source chunks used by the chain is indeed from that doc that contains the correct answer (**13,500**), other source chunks from different docs are included, and the answer is therefore incorrect.

In [26]:
chain_response["source_documents"]

[Document(page_content='1.1 Landlord. DHA Group , a Delaware limited liability company authorized to transact business in New Jersey .', metadata={'xpath': '/docset:OFFICELEASE-section/docset:OFFICELEASE-section/docset:OFFICELEASE/docset:WITNESSETH-section/docset:WITNESSETH/dg:chunk/dg:chunk/docset:BasicLeaseInformation/docset:BASICLEASEINFORMATIONANDDEFINEDTERMS-section/docset:BASICLEASEINFORMATIONANDDEFINEDTERMS/docset:DhaGroup/dg:chunk[1]/docset:Landlord-section/dg:chunk[2]', 'id': 'voxlha0preyt', 'name': 'Sample Commercial Leases/Shorebucks LLC_NJ.pdf', 'source': 'Sample Commercial Leases/Shorebucks LLC_NJ.pdf', 'structure': 'lim h1 div', 'tag': 'chunk', 'Lease Date': 'March  29th , 2019', 'Landlord': 'DHA Group', 'Tenant': 'Shorebucks LLC'}),
 Document(page_content="1.16 Landlord's Notice Address. DHA Group , Suite 1010 , 111 Bauer Dr , Oakland , New Jersey , 07436 , with a copy to the Building Management Office at the Project, Attention: On-Site Property Manager.", metadata={'xpa

Docugami can help here. Chunks are annotated with additional metadata created using different techniques if a user has been [using Docugami](https://help.docugami.com/home/reports). More technical approaches will be added later.

Specifically, let's look at the additional metadata that is returned on the documents returned by docugami, in the form of some simple key/value pairs on all the text chunks:

In [27]:
loader = DocugamiLoader(docset_id="zo954yqy53wp")
documents = loader.load()
documents[0].metadata

{'xpath': '/docset:OFFICELEASE-section/docset:OFFICELEASE-section/dg:chunk/docset:Lease',
 'id': 'cpzwzcurck2t',
 'name': 'Sample Commercial Leases/Shorebucks LLC_AZ.pdf',
 'source': 'Sample Commercial Leases/Shorebucks LLC_AZ.pdf',
 'structure': 'h1 h1 p',
 'tag': 'chunk Lease',
 'Lease Date': 'March  29th , 2019',
 'Landlord': 'Menlo Group',
 'Tenant': 'Shorebucks LLC'}

We can use a [self-querying retriever](/docs/modules/data_connection/retrievers/how_to/self_query/) to improve our query accuracy, using this additional metadata:

In [30]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.vectorstores import Chroma
import chromadb

EXCLUDE_KEYS = ["id", "xpath", "structure"]
metadata_field_info = [
    AttributeInfo(
        name=key,
        description=f"The {key} for this chunk",
        type="string",
    )
    for key in documents[0].metadata
    if key.lower() not in EXCLUDE_KEYS
]


document_content_description = "Contents of this chunk"
llm = OpenAI(temperature=0)

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("docugami_test")
vectordb = Chroma.from_documents(documents=documents, embedding=embedding, collection=collection)

retriever = SelfQueryRetriever.from_llm(
    llm, vectordb, document_content_description, metadata_field_info, verbose=True
)
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True
)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_20678/3046728812.py", line 4, in <module>
    import chromadb
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/chromadb/__init__.py", line 4, in <module>
    import chromadb.config
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/chromadb/config.py", line 1, in <module>
    from pydantic import BaseSettings
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/pydantic/__init__.py", line 218, in __getattr__
    return _getattr_migration(attr_name)
  File "/root/Source/github/docugami.langchain/libs/langchain/.venv/lib/python3.9/site-packages/pydantic/_migration.py", line 294, in wrapper
    

Let's run the same question again. It returns the correct result since all the chunks have metadata key/value pairs on them carrying key information about the document even if this information is physically very far away from the source chunk used to generate the answer.

In [ ]:
qa_chain(
    "What is rentable area for the property owned by DHA Group?"
)  # correct answer should be 13,500 sq ft

This time the answer is correct, since the self-querying retriever created a filter on the landlord attribute of the metadata, correctly filtering to document that specifically is about the DHA Group landlord. The resulting source chunks are all relevant to this landlord, and this improves answer accuracy even though the landlord is not directly mentioned in the specific chunk that contains the correct answer.